In [1]:
#@markdown Configuración Inicial
from IPython.utils import io

from google.colab.data_table import DataTable
from IPython.display import Javascript
from IPython.display import display, Markdown, Latex
from IPython.display import Audio, Image
from IPython.display import IFrame, HTML
with io.capture_output() as capt:
  !pip install gradio

import gradio as gr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

def dLatex(self):
  return display(Latex(self))

def dMarkdown(self):
  return display(Markdown(self))

# Avoids scroll-in-the-scroll in the entire Notebook
# https://stackoverflow.com/a/66891328
def resize_colab_cell():
  display(Javascript(
      'google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'
      ))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

def dLatex(self):
  return display(Latex(self))

def dMarkdown(self):
  return display(Markdown(self))

# Corrección de aire libre

In [2]:
#@markdown Corrección de aire libre:
#@markdown $$0.3086\,h\,\mathrm{(mGal)}$$

def fag(h):
  return 0.3086*h #mGal

#@markdown Corrección simple de Bouguer


#@markdown $$2\pi{G}\rho{h}\,\mathrm{(mGal)}$$
def bouguer(h, rho=2.670):
  G=6.674e-3
  return 2*np.pi*h

<IPython.core.display.Javascript object>

# Deriva Instrumental

Deriva *(drift)*: cambios con el tiempo en la lectura de los gravímetros en una estación

**Causas:**
* Constante elástica del resorte (resortes y fibras de torsión no son perfectamente elásticos)
* Temperatura
* Mareas terrestres

Al repetir una medición en el mismo lugar y bajo las mismas circunstancias, no  se obtiene el mismo valor (variaciones de aprox. $0.001\,\mathrm{mGal}$)

**¿De qué depende la deriva?**
* Tipo de instrumento (varía de un instrumento a otro)
* Características del resorte
* Edad y uso del gravímetro
* Fluctuaciones de temperatura externa
* Tipo de transporte
* Cambios de presión
* Cambios de voltaje en la batería
* Valores de deriva suelen ser menores a $1\,\mathrm{mGal}$


**Deriva según el gravímetro:**

* Gravímetros LC&R:
  * Instrumentos nuevos en promedio menor a $1\,\mathrm{mGal/mes}$
  * en un equipo con algunos años de uso, deriva menor a $0.5\,\mathrm{mGal/mes}$ (Manual LC&R)
* Gravímetros Scintrex: Modelo CG5, deriva residual menor a $20\,\mu\mathrm{Gal/dia}$
  * El instrumento realiza correcciones automáticas
* Gravímetros Burries deriva menor a $0.3\,\mathrm{mGal/mes}$ (Jentzsch et al., 2017)

**¿Cómo se corrige la deriva?**
* Realizar mediciones en la estación base con diferencia
de una o dos horas
* Graficar las lecturas en función del tiempo
* Por ello, primera y última medición del día se realizan
en la estación base



\begin{align}
d
&=
\frac{L_{f}-L_{i}}{t_{f}-t_{i}}
\\
{L^{c}}_{j}
&=
L_{j}-d\left(t_{j}-t_{A i}\right)
\end{align}

In [3]:
k = 0.877 #@param {type:"slider", min:0.5, max:1, step:0.001}
g_0 = 956694 #@param {type:"slider", min:9e5, max:10e5, step:1}
#@markdown Tabla

pd.options.display.float_format = '{:.2f}'.format
df=pd.DataFrame([[ 1, "08:00", 520],
                 [ 2, "08:30", 490],
                 [ 3, "09:05", 450],
                 [ 4, "09:40", 420],
                 [ 1, "10:10", 522],
                 [ 5, "10:45", 470],
                 [ 6, "11:20", 510],
                 [ 7, "11:55", 580],
                 [ 4, "12:40", 424],
                 [ 8, "13:40", 570],
                 [ 9, "14:15", 650],
                 [10, "14:50", 510],
                 [ 7, "15:15", 575]],
                columns=["ESTACIÓN", "Hora", "Lectura"]
                )

@np.vectorize
def hour2min(hour):
  h, m = hour.split(':')
  return 60*int(h)+int(m)

df['Minutos'] = hour2min(df['Hora'])
df0 = df.copy()

display(df0.set_index("ESTACIÓN").T)

<IPython.core.display.Javascript object>

ESTACIÓN,1,2,3,4,1,5,6,7,4,8,9,10,7
Hora,08:00,08:30,09:05,09:40,10:10,10:45,11:20,11:55,12:40,13:40,14:15,14:50,15:15
Lectura,520,490,450,420,522,470,510,580,424,570,650,510,575
Minutos,480,510,545,580,610,645,680,715,760,820,855,890,915


In [4]:
k = 0.882
g_= 979166.3

<IPython.core.display.Javascript object>

In [5]:
def find_ran(df, value):
  dfd = df.loc[df['ESTACIÓN'] == value]
  min, max = tuple(dfd.index)[:2]
  return np.arange(min, max+1)

def rev_loop(dfv, v, g):
  ind = dfv.index
  delta_l, delta_m = tuple(dfv[['Lectura','Minutos']].\
                           iloc[[0,-1]].diff().iloc[-1])
  d = delta_l/delta_m
  dfv['Lectura_rev'] = [df['Lectura'][i] - 
                        d * (df['Minutos'][i] -
                             df['Minutos'][min(ind)])
                        for i in ind]
  dfv['delta_g'] = k * dfv['Lectura_rev'].diff().fillna(0)
  dfv['g'] = g + dfv['delta_g'].cumsum()
  return d, dfv

<IPython.core.display.Javascript object>

In [6]:
df_1 = df.loc[find_ran(df, 1)]
d_1, df_1 = rev_loop(df_1, 1, g_0)
dMarkdown(f'$d={d_1:g}$')
display(df_1)

<IPython.core.display.Javascript object>

$d=0.0153846$

,ESTACIÓN,Hora,Lectura,Minutos,Lectura_rev,delta_g,g
0,1,08:00,520,480,520.00,0.00,956694.00
1,2,08:30,490,510,489.54,-26.87,956667.13
2,3,09:05,450,545,449.00,-35.75,956631.38
3,4,09:40,420,580,418.46,-26.93,956604.44
4,1,10:10,522,610,520.00,89.56,956694.00


In [7]:
lrev_4 = df_1['Lectura_rev'].iloc[-2]
g_4 = df_1['g'].iloc[-2]
print(lrev_4, g_4, sep='\n')

df_4 = df.loc[np.delete(find_ran(df, 4), 1)]
df.loc[min(df_4.index), 'Lectura'] = lrev_4

d_4, df_4 = rev_loop(df_4, 4, g_0)
dMarkdown(f'$d={d_4:g}$')
display(df_4)

<IPython.core.display.Javascript object>

418.46153846153845
956604.443076923


$d=0.0222222$

,ESTACIÓN,Hora,Lectura,Minutos,Lectura_rev,delta_g,g
3,4,09:40,420,580,418.46,0.00,956694.00
5,5,10:45,470,645,468.56,44.18,956738.18
6,6,11:20,510,680,507.78,34.59,956772.78
7,7,11:55,580,715,577.00,61.05,956833.83
8,4,12:40,424,760,420.00,-138.47,956695.36


In [8]:
lrev_7 = df_4['Lectura_rev'].iloc[-2]
g_7 = df_4['g'].iloc[-2]
print(lrev_7, g_7, sep='\n')

df_7 = df.loc[np.delete(find_ran(df, 7), 4)]
df.loc[min(df_7.index), 'Lectura'] = lrev_7

d_7, df_7 = rev_loop(df_7, 7, g_0)
dMarkdown(f'$d={d_7:g}$')
display(df_7)

<IPython.core.display.Javascript object>

577.0
956833.830923077


$d=-0.025$

,ESTACIÓN,Hora,Lectura,Minutos,Lectura_rev,delta_g,g
7,7,11:55,580.00,715,577.00,0.00,956694.00
8,4,12:40,424.00,760,425.12,-133.95,956560.05
9,8,13:40,570.00,820,572.62,130.09,956690.14
10,9,14:15,650.00,855,653.50,71.33,956761.47
12,7,15:15,575.00,915,580.00,-64.83,956696.65


In [10]:
display(df_1)
display(df_4)
display(df_7)

df_rev = pd.concat([
           df_1.iloc[:-1],
           df_4.iloc[1:-1],
           df_7.iloc[1:-1]
])

df_rev['diff'] = (k * df_rev['Lectura_rev'].diff().fillna(0) - 
                      df_rev['delta_g'])
display(df_rev)
df_rev['diff'].sum()

<IPython.core.display.Javascript object>

,ESTACIÓN,Hora,Lectura,Minutos,Lectura_rev,delta_g,g
0,1,08:00,520,480,520.00,0.00,956694.00
1,2,08:30,490,510,489.54,-26.87,956667.13
2,3,09:05,450,545,449.00,-35.75,956631.38
3,4,09:40,420,580,418.46,-26.93,956604.44
4,1,10:10,522,610,520.00,89.56,956694.00


,ESTACIÓN,Hora,Lectura,Minutos,Lectura_rev,delta_g,g
3,4,09:40,420,580,418.46,0.00,956694.00
5,5,10:45,470,645,468.56,44.18,956738.18
6,6,11:20,510,680,507.78,34.59,956772.78
7,7,11:55,580,715,577.00,61.05,956833.83
8,4,12:40,424,760,420.00,-138.47,956695.36


,ESTACIÓN,Hora,Lectura,Minutos,Lectura_rev,delta_g,g
7,7,11:55,580.00,715,577.00,0.00,956694.00
8,4,12:40,424.00,760,425.12,-133.95,956560.05
9,8,13:40,570.00,820,572.62,130.09,956690.14
10,9,14:15,650.00,855,653.50,71.33,956761.47
12,7,15:15,575.00,915,580.00,-64.83,956696.65


,ESTACIÓN,Hora,Lectura,Minutos,Lectura_rev,delta_g,g,diff
0,1,08:00,520.00,480,520.00,0.00,956694.00,0.00
1,2,08:30,490.00,510,489.54,-26.87,956667.13,0.00
2,3,09:05,450.00,545,449.00,-35.75,956631.38,0.00
3,4,09:40,420.00,580,418.46,-26.93,956604.44,0.00
5,5,10:45,470.00,645,468.56,44.18,956738.18,0.00
6,6,11:20,510.00,680,507.78,34.59,956772.78,0.00
7,7,11:55,580.00,715,577.00,61.05,956833.83,0.00
8,4,12:40,424.00,760,425.12,-133.95,956560.05,0.00
9,8,13:40,570.00,820,572.62,130.09,956690.14,0.00
10,9,14:15,650.00,855,653.50,71.33,956761.47,0.00


0.0